In [ ]:
!pip install transformers
!pip install datsets transformers[sentencepiece]
!pip install sentencepiece
!pip install --quiet pytorch-lightning
!pip install --quiet torchtext
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement datsets (from versions: none)
ERROR: No matching distribution found for datsets
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 4.7 MB/s eta 0:00:00


In [ ]:
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
import torch

topic_trained_model_path = '/content/drive/MyDrive/fyp/model'
topic_trained_tokenizer = '/content/drive/MyDrive/fyp/tokenizer'

topic_model = T5ForConditionalGeneration.from_pretrained(topic_trained_model_path)
topic_tokenizer = T5Tokenizer.from_pretrained(topic_trained_tokenizer)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
topic_model = topic_model.to(device)

def topic_modeling(text) :
    topic = ""
    encoding = topic_tokenizer.encode_plus(text,max_length =512, padding=True, return_tensors="pt")
    input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    beam_outputs = topic_model.generate(
        input_ids=input_ids,attention_mask=attention_mask,
        max_length=72,
        early_stopping=True,
        num_beams=5,
        num_return_sequences=1

    )

    for beam_output in beam_outputs:
        sent = topic_tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        topic = sent

    topic = topic.split(':')[1]
    print (topic)
    return topic

device  cpu


In [ ]:
!pip install Keras-Preprocessing


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.6 MB/s eta 0:00:00


In [ ]:
from tensorflow.keras.models import load_model
from keras_preprocessing.sequence import pad_sequences
import pickle
SEQUENCE_LENGTH =300
sentiment_trained_model_path = '/content/drive/MyDrive/fyp/final/model.h5'
sentiment_trained_tokenizer_path = '/content/drive/MyDrive/fyp/final/tokenizer.pkl'

sentiment_model = load_model(sentiment_trained_model_path)
# Load the tokenizer
with open(sentiment_trained_tokenizer_path, 'rb') as f:
    sentiment_tokenizer = pickle.load(f)

def sentiment_score_prediction(text):
    x_test = pad_sequences(sentiment_tokenizer.texts_to_sequences([text]), maxlen=SEQUENCE_LENGTH)
    score = sentiment_model.predict([x_test])[0]
    return score[0]

In [ ]:
text = "A provision of the U.S. debt ceiling bill that streamlined the federal approval process for the $6.6 billion Mountain Valley Pipeline and limited court reviews of challenges to the project violates the U.S. Constitution’s separation of powers doctrine, opponents of the pipeline have claimed.Today, after months and years being held up the government just released this: 'Mountain Valley is authorized by the Commission to proceed with all construction activities at all locations in accordance with federal authorizations, notwithstanding any request for rehearing of this order.' Mountain Valley Pipeline LLC - a joint project between Equitrans, NextEra Energy Inc., Consolidated Edison Inc., AltaGas Ltd and RGC Resources Inc. - has said the pipeline is over 94% complete.VERY BULLISH!"
sentiment_score = sentiment_score_prediction(topic_modeling(text))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


 Mountain Valley Pipeline LLC - a joint project between Equitrans, NextEra Energy Inc., Consolidated Edison Inc., AltaGas Ltd. - has said the pipeline is over 94% complete.
1/1 [==============================] - 0s 133ms/step


In [ ]:
print (sentiment_score)

0.9064671


In [ ]:
sentiment_score = sentiment_score_prediction("$ETRN Equitrans has amazing value right now")

1/1 [==============================] - 1s 766ms/step


In [ ]:
print (sentiment_score)

0.9064671


In [ ]:
import pandas as pd

# Step 1: Import the required libraries

# Step 2: Read the CSV file
data = pd.read_csv('/content/drive/MyDrive/fyp/dataset/extracted_btc.csv')

# Step 3: Create an empty list to store the sentiment scores
sentiment_scores = []

# Step 4: Iterate over each row in the CSV file
for index, row in data.iterrows():
    body_text = row['text']


    score = sentiment_score_prediction(body_text)
    sentiment_scores.append(score)

# Step 5: Add the sentiment scores as a new column in the DataFrame
data['sentiment_score'] = sentiment_scores

# Step 6: Save the updated DataFrame to a new CSV file
data.to_csv('output_csv_file_btc.csv', index=False)

# Step 6: Save the updated DataFrame to a new CSV file
output_file = 'output_csv_file_btc.csv'
data.to_csv(output_file, index=False)

# Step 7: Download the CSV file
from google.colab import files
files.download(output_file)


1/1 [==============================] - 0s 78ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Step 1: Import the required libraries

# Step 2: Read the CSV file
data = pd.read_csv('/content/drive/MyDrive/fyp/dataset/extracted.csv')

# Step 3: Create an empty list to store the sentiment scores
sentiment_scores = []

# Step 4: Iterate over each row in the CSV file
for index, row in data.iterrows():
    body_text = row['text']


    score = sentiment_score_prediction(body_text)
    sentiment_scores.append(score)

# Step 5: Add the sentiment scores as a new column in the DataFrame
data['sentiment_score'] = sentiment_scores

# Step 6: Save the updated DataFrame to a new CSV file
data.to_csv('output_csv_file.csv', index=False)

# Step 6: Save the updated DataFrame to a new CSV file
output_file = 'output_csv_file.csv'
data.to_csv(output_file, index=False)

# Step 7: Download the CSV file
from google.colab import files
files.download(output_file)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 85ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Step 1: Import the required libraries

# Step 2: Read the CSV file
data = pd.read_csv('/content/drive/MyDrive/fyp/dataset/extracted.csv')

# Step 3: Create an empty list to store the sentiment scores
sentiment_scores = []

# Step 4: Iterate over each row in the CSV file
for index, row in data.iterrows():
    body_text = row['preprocessed_body']
    title_text = row['preprocessed_title']

    # Concatenate body and title if desired
    text = f"{body_text} {title_text}"

    score = sentiment_score_prediction(text)
    sentiment_scores.append(score)

# Step 5: Add the sentiment scores as a new column in the DataFrame
data['sentiment_score'] = sentiment_scores

# Step 6: Save the updated DataFrame to a new CSV file
data.to_csv('output_csv_file.csv', index=False)

# Step 6: Save the updated DataFrame to a new CSV file
output_file = 'output_csv_file.csv'
data.to_csv(output_file, index=False)

# Step 7: Download the CSV file
from google.colab import files
files.download(output_file)


1/1 [==============================] - 0s 136ms/step


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 12>:19                                                                            │
│ in sentiment_score_prediction:15                                                                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py:65 in error_handler       │
│                                                                                                  │
│    62 │   │                                                                                      │
│    63 │   │   filtered_tb = None                                                                 │
│    64 │   │   try:                                                                               │
│ ❱  65 │   │   │   return fn(*args, **kwargs)                                                     │
│    66 │   │   except Exception as e:                                                             │
│    67 │   │   │   filtered_tb = _process_traceback_frames(e.__traceback__)                       │
│    68 │   │   │   # To get the full stack trace, call:                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/keras/engine/training.py:2349 in predict                 │
│                                                                                                  │
│   2346 │   │   │   │   │   │   stacklevel=2,                                                     │
│   2347 │   │   │   │   │   )                                                                     │
│   2348 │   │   │                                                                                 │
│ ❱ 2349 │   │   │   data_handler = data_adapter.get_data_handler(                                 │
│   2350 │   │   │   │   x=x,                                                                      │
│   2351 │   │   │   │   batch_size=batch_size,                                                    │
│   2352 │   │   │   │   steps_per_epoch=steps,                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/keras/engine/data_adapter.py:1583 in get_data_handler    │
│                                                                                                  │
│   1580 │   """                                                                                   │
│   1581 │   if getattr(kwargs["model"], "_cluster_coordinator", None):                            │
│   1582 │   │   return _ClusterCoordinatorDataHandler(*args, **kwargs)                            │
│ ❱ 1583 │   return DataHandler(*args, **kwargs)                                                   │
│   1584                                                                                           │
│   1585                                                                                           │
│   1586 def _make_class_weight_map_fn(class_weight):                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/keras/engine/data_adapter.py:1260 in __init__            │
│                                                                                                  │
│   1257 │   │   │   self._steps_per_execution = steps_per_execution                               │
│   1258 │   │                                                                                     │
│   1259 │   │   adapter_cls = select_data_adapter(x, y)                                           │
│ ❱ 1260 │   │   self._adapter = adapter_cls(                